In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
from tensorflow.python.keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import tensorflow as tf
import gc

In [ ]:
# Loading data and putting them into training and test sets

#locations setting for training and test datasets
train_data='/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Training'
test_data='/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test'

In [ ]:
 
values = tf.io.read_file('/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test/cucumber_1/r0_79.jpg')


In [ ]:
#creates X_train and Y_train using file_names and folders
def load_f(path):
    data = load_files(path)
    return data
def get_data(path):
    data = load_f(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    
    return files,targets,target_labels

X_train, Y_train, labels = get_data(train_data)
X_test, Y_test,_ = get_data(test_data)
Y_train = np_utils.to_categorical(Y_train, 120)
Y_test = np_utils.to_categorical(Y_test, 120)

In [ ]:
def decode_image(filename, image_type, resize_shape, channels):
    value = tf.io.read_file(filename)
    if image_type == 'png':
        decoded_image = tf.image.decode_png(value, channels=channels)
    elif image_type == 'jpeg':
        decoded_image = tf.image.decode_jpeg(value, channels=channels)
    else:
        decoded_image = tf.image.decode_image(value, channels=channels)
    
    if resize_shape is not None and image_type in ['png', 'jpeg']:
        decoded_image = tf.image.resize(decoded_image, resize_shape)
    
    return decoded_image

In [ ]:

def get_dataset(image_paths, image_type, resize_shape, channels):
    filename_tensor = tf.constant(image_paths)
    dataset = tf.data.Dataset.from_tensor_slices(filename_tensor)
    
    def _map_fn(filename):
        decode_images = decode_image(filename, image_type, resize_shape, channels=channels)
        return decode_images
    
    map_dataset = dataset.map(_map_fn) # we use the map method: allow to apply the function _map_fn to all the 
    # elements of dataset 
    return map_dataset

In [ ]:
def get_image_data(image_paths, image_type, resize_shape, channels):
    dataset = get_dataset(image_paths, image_type, resize_shape, channels)
    iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
    next_image = iterator.get_next()
    
    return next_image

In [ ]:
X_train, X_val = train_test_split(X_train, test_size=0.2, random_state=33)
Y_train, Y_val = train_test_split(Y_train, test_size=0.2, random_state=33)

In [ ]:
X_train

In [ ]:
def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        images_as_array.append(img_to_array(load_img(file)).astype(np.float16))
        gc.collect()
        #print(images_as_array[-1])
    return images_as_array

X_train = np.array(get_image_data(X_train,'jpg',0,0))

In [ ]:
X_train

In [ ]:

X_val = (get_image_data(X_val,'jpg',0,0))
X_test =(get_image_data(X_test,'jpg',0,0))
z=get_image_data(['/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test/cucumber_1/r0_95.jpg'],'jpg',0,0)

In [ ]:
z=get_image_data('/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test/cucumber_1/r0_95.jpg','jpg',0,0)
z

In [ ]:
train_ds = tf.data.Dataset.list_files(X_train, shuffle=False)

In [ ]:
valid_ds=tf.data.Dataset.list_files(X_val, shuffle=False)
test_ds=tf.data.Dataset.list_files(X_test, shuffle=False)

In [ ]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img,label

In [ ]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [ ]:
image=process_image(b'/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Training/apple_6/r0_14.jpg')
image

In [ ]:
images_ds = tf.data.Dataset.list_files('/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Training/*/*', shuffle=False)
image_count=len(images_ds)
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [ ]:
image = tf.cast(image,'float32')
plt.imshow(image.eval())
plt.show()
# images_ds = images_ds.shuffle(200)
# for file in images_ds.take(3):
#     print(file.numpy())

In [ ]:
train_ds = train_ds.map(process_image)
#valid_ds = valid_ds.map(process_image)
test_ds  = test_ds.map(process_image)

In [ ]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

In [ ]:
import gc
gc.collect()


In [ ]:
X_train = X_train.astype('float32')/255
X_val = X_val.astype('float32')/255
X_test = X_test.astype('float32')/255

In [4]:
# Load training and validation sets
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory
ds_train_ = image_dataset_from_directory(
    '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training',
    
    validation_split=0.2,
    subset="training",
    seed=13,
    labels='inferred',
    label_mode='categorical',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training',
    validation_split=0.2,
    subset="validation",
    seed=13,
    labels='inferred',
    label_mode='categorical',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [9]:
import pathlib
data_dir='/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
data_dir=pathlib.Path(data_dir)

In [21]:
class_names = (sorted([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"]))
print(class_names)

In [ ]:
for image, label in ds_train.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
from tensorflow import keras
from tensorflow.python.keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import tensorflow as tf
import gc

from sklearn.model_selection import RandomizedSearchCV, KFold,GridSearchCV


In [ ]:
pip install --upgrade tensorflow


In [5]:
import tensorflow as tf
# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
 #tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
# tf.config.experimental_run_functions_eagerly(True)
from tensorflow import keras
from tensorflow.keras import layers




model = keras.Sequential([

# Attach a global average pooling layer after the base
layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=[128, 128, 3]),
layers.experimental.preprocessing.RandomRotation(0.1),
layers.experimental.preprocessing.RandomZoom(0.1),
layers.experimental.preprocessing.Rescaling(1./255, input_shape=([128, 128, 3])),
layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
              input_shape=[128, 128, 3]),
layers.MaxPool2D(),

# Block Two
layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
layers.MaxPool2D(),

# Block Three
# YOUR CODE HERE
layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
layers.MaxPool2D(),

# Head
layers.Flatten(),
layers.Dense(30, activation='relu'),
layers.Dropout(0.2),
layers.Dense(30, activation='relu'),
layers.Dropout(0.2),
layers.Dense(131, activation='sigmoid')
])

model.compile(
optimizer='adam',
# YOUR CODE HERE: Add loss and metric
loss='categorical_crossentropy',
metrics=['accuracy']
)
 
    


In [ ]:
import tensorflow as tf
# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
#tf.config.experimental_run_functions_eagerly(True)
from tensorflow import keras
from tensorflow.keras import layers



# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():  
    model = keras.Sequential([

# Attach a global average pooling layer after the base
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=[128, 128, 3]),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=([128, 128, 3])),
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                  input_shape=[128, 128, 3]),
    layers.MaxPool2D(),

    # Block Two
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Three
    # YOUR CODE HERE
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Head
    layers.Flatten(),
    layers.Dense(30, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(30, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(131, activation='sigmoid')
    ])

    model.compile(
    optimizer='adam',
    # YOUR CODE HERE: Add loss and metric
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

In [ ]:
model.summary()

In [7]:
history=model.fit(ds_train,
        batch_size = 128,
        epochs=3,
        validation_data=ds_valid,
        verbose=2,shuffle=True)

In [ ]:
from keras.models import load_model


model.save('my_model.h5')
    


In [ ]:
from matplotlib.pyplot import plot
import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()

In [ ]:
ds_test_=image_dataset_from_directory(
    '/kaggle/input/fruits/fruits-360-original-size/fruits-360-original-size/Test',
    labels='inferred',
    label_mode='categorical',
    image_size=[128, 128],
    interpolation='nearest',
    shuffle=True,
)
ds_test = (
    ds_test_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
def create_model(layers,activation):
    model=keras.sequential()
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(RandomFlip("horizontal",
                      input_shape=[128, 128, 3]))
            model.add(RandomRotation(0.1))
            model.add(RandomZoom(0.1))
            model.add(Rescaling(1./255, input_shape=([128, 128, 3])))
            model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                          input_shape=[128, 128, 3]))
            model.add(MaxPool2D())

            # Block Two
            model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))
            model.add(MaxPool2D())

            # Block Three
            # YOUR CODE HERE
            model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
            model.add(Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))
            model.add(MaxPool2D())

            # Head
            model.add(Flatten())
        else :
            model.add(dense(nodes, activation='relu'))
            layers.Dropout(0.2)
        model.add(dense(131, activation='sigmoid'))    
    model.compile(
    optimizer='adam',
    # YOUR CODE HERE: Add loss and metric
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
    return model
model=tf.keras.wrappers.scikit_learn.KerasClassifier(batch_size = 128,
        validation_data=ds_valid,verbose=2, shuffle=True,build_fn=create_model)


In [ ]:
layers=[[32],[64,32,32],[64,32]]
param = dict(layers=layers,epochs=[10])
X,y=ds_train.next()

rf_random = GridSearchCV(estimator = model,param_grid = param)
grid_result=rf_random.fit(X,y)

In [ ]:
img = tf.keras.utils.load_img('/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/Pineapple Mini/r_208_100.jpg'
    , target_size=(128, 128)
)
img_array = convert_to_float(img,'Pineapple')
img_array

In [17]:
img = tf.keras.utils.load_img('/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test/Chestnut/r2_116_100.jpg'
    , target_size=(128, 128,3)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 
#img_array=tf.keras.layers.Rescaling(1./255)(img_array)
print(img_array)
predictions=model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)
print(score)
# for image, label in ds_test.take(1):
#     #print("****Image: ",image.numpy()[0][0])
#      print("****Label: ",label.numpy())